Blok 1: Kütüphaneler ve Dosya Yolları

Önce temel araçlarımızı çağıralım ve yolları tanımlayalım.

In [5]:
import os
import cv2
import numpy as np
import tensorflow as tf
# Keras'ı doğrudan tf üzerinden çağıralım (Pylance bunu daha kolay anlar)
from tensorflow import keras
layers = keras.layers
models = keras.models
applications = keras.applications

import matplotlib.pyplot as plt
import sklearn.metrics as metrics # sklearn kuruluysa sarı çizgi gitmeli

# Dosya yolları
LIDAR_PATH = r"C:\Users\alkay\ITU_C\chactun\data\Chactun_ML_ready_lidar\lidar"
MASK_PATH = r"C:\Users\alkay\ITU_C\chactun\data\Chactun_ML_ready_masks\masks"

# Eğitim ve Test sınırları
TRAIN_END = 1764
TEST_END = 2093

Blok 2: Veri Yükleme Fonksiyonu

Bu blokta .tif dosyalarını okuyup, maskeleri birleştirip (3 kanallı hale getirip) normalleştireceğiz.

In [ ]:
def build_vgg19_unet(input_shape=(224, 224, 1)):
    inputs = layers.Input(shape=input_shape)
    
    # 1 kanallı LiDAR verisini VGG19'un beklediği 3 kanallı yapıya çoğaltıyoruz
    x_3ch = layers.Concatenate()([inputs, inputs, inputs])
    
    # Encoder: Önceden eğitilmiş VGG19
    vgg19 = applications.VGG19(include_top=False, weights='imagenet', input_tensor=x_3ch)
    
    # VGG19'dan Skip Connection (Atlama Bağlantıları) için çıkışları alıyoruz
    s1 = vgg19.get_layer("block1_conv2").output # (224, 224, 64)
    s2 = vgg19.get_layer("block2_conv2").output # (112, 112, 128)
    s3 = vgg19.get_layer("block3_conv4").output # (56, 56, 256)
    s4 = vgg19.get_layer("block4_conv4").output # (28, 28, 512)
    
    # Bridge (Köprü)
    b1 = vgg19.get_layer("block5_conv4").output # (14, 14, 512)
    
    # Decoder (Çözücü) Katmanları
    # UpBlock 1
    d1 = layers.UpSampling2D((2, 2))(b1)
    d1 = layers.concatenate([d1, s4])
    d1 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(d1)
    d1 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(d1)
    
    # UpBlock 2
    d2 = layers.UpSampling2D((2, 2))(d1)
    d2 = layers.concatenate([d2, s3])
    d2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(d2)
    d2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(d2)
    
    # UpBlock 3
    d3 = layers.UpSampling2D((2, 2))(d2)
    d3 = layers.concatenate([d3, s2])
    d3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(d3)
    d3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(d3)
    
    # UpBlock 4
    d4 = layers.UpSampling2D((2, 2))(d3)
    d4 = layers.concatenate([d4, s1])
    d4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(d4)
    d4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(d4)
    
    # Çıkış Katmanı: 3 maske sınıfı var, bağımsız olasılıklar için Sigmoid kullanıyoruz
    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(d4)
    
    model = models.Model(inputs=[inputs], outputs=[outputs], name="VGG19_UNET")
    return model

# Modeli oluştur
model = build_vgg19_unet()

# Optimizer ve Loss fonksiyonu ayarları
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                  loss='binary_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=2)])

print("VGG-19 tabanlı U-Net başarıyla oluşturuldu!")
model.summary()

80134624/80134624 [==============================] - 35s 0us/step
VGG-19 tabanlı U-Net başarıyla oluşturuldu!
Model: "VGG19_UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 concatenate (Concatenate)      (None, 224, 224, 3)  0           ['input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]']                
                                                                              

In [3]:
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"✅ GPU Algılandı: {gpu_devices}")
else:
    print("❌ GPU Algılanamadı, CPU kullanılacak.")

✅ GPU Algılandı: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Blok 3: VGG-19 Tabanlı U-Net Modeli

VGG-19'un ilk katmanlarını "Encoder" (özellik çıkarıcı) olarak kullanacağız. Segmentasyon yaptığımız için görüntüyü tekrar eski boyutuna getiren bir "Decoder" eklememiz gerekiyor.

In [4]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# En iyi modelin kaydedileceği klasörü oluşturalım
os.makedirs("checkpoints", exist_ok=True)

# Eğitim Asistanları (Callbacks)
# 1. Her epoch sonunda sadece en iyi modeli (val_loss'a göre) kaydeder
checkpoint = ModelCheckpoint(
    filepath="checkpoints/vgg19_unet_best.h5",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

# 2. Model 10 tur boyunca ilerleme kaydetmezse eğitimi erken bitirir
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# 3. Eğitim duraklarsa öğrenme oranını (learning rate) düşürerek ince ayar yapar
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

# Eğitimi Başlatıyoruz!
EPOCHS = 50
print("Eğitim süreci başlatılıyor... RTX 4060 devreye giriyor 🚀")

history = model.fit(
    train_gen,                   # Blok 2'deki eğitim jeneratörü
    validation_data=test_gen,    # Blok 2'deki test (doğrulama) jeneratörü
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stop, reduce_lr],
    verbose=1
)

Eğitim süreci başlatılıyor... RTX 4060 devreye giriyor 🚀


NameError: name 'train_gen' is not defined

Blok 4: Eğitim (Training)

Modeli eğitmeye başlayalım. Arkeolojik verilerde nesneler seyrek olduğu için binary_crossentropy başlangıç için iyidir.

In [ ]:
history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=25, 
    batch_size=16
)

# Eğitim sonucunu görselleştir
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.legend()
plt.show()

### Analizler

1. Sayısal Analiz: IoU (Intersection over Union) Skoru
Segmentasyonun "altın standardı" IoU (Kesişim bölü Birleşim) değeridir. Bu, modelin tahmin ettiği maske ile gerçek maskenin ne kadar üst üste bindiğini ölçer.

In [ ]:
from sklearn.metrics import jaccard_score

def calculate_iou(model, X_test, y_test):
    y_pred = model.predict(X_test)
    # Tahminleri 0.5 eşiğiyle 0 veya 1'e çevirelim
    y_pred_thresholded = (y_pred > 0.5).astype(np.uint8)
    
    classes = ['Building', 'Aguada', 'Platform']
    ious = []
    
    print("\n--- SINIF BAZLI IoU SKORLARI ---")
    for i, class_name in enumerate(classes):
        # Her sınıf için IoU hesapla
        iou = jaccard_score(y_test[..., i].flatten(), y_pred_thresholded[..., i].flatten(), pos_label=1)
        ious.append(iou)
        print(f"{class_name} IoU: {iou:.4f}")
    
    print(f"\nOrtalama (Mean) IoU: {np.mean(ious):.4f}")

# Modeli test verisiyle değerlendir
calculate_iou(model, X_test, y_test)

2. Görselleştirme: Tahminleri Karşılaştırma
Modelin neyi doğru, neyi yanlış bildiğini görmenin en iyi yolu; Orijinal LiDAR, Gerçek Maske ve Modelin Tahminini yan yana koymaktır.

In [ ]:
def visualize_results(model, X_test, y_test, num_samples=3):
    # Rastgele örnekler seç
    indices = np.random.randint(0, len(X_test), num_samples)
    y_pred = model.predict(X_test[indices])
    y_pred_thresholded = (y_pred > 0.5).astype(np.uint8)

    plt.figure(figsize=(15, 5 * num_samples))
    
    for i, idx in enumerate(indices):
        # 1. Sütun: Orijinal LiDAR
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(X_test[idx].squeeze(), cmap='gray')
        plt.title(f"Örnek {idx}: Orijinal LiDAR")
        plt.axis('off')

        # 2. Sütun: Gerçek Maske (Ground Truth)
        # Sınıfları renkli görmek için 3 kanalı da gösteriyoruz
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(y_test[idx])
        plt.title("Gerçek Maske")
        plt.axis('off')

        # 3. Sütun: Modelin Tahmini
        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(y_pred_thresholded[i])
        plt.title("Model Tahmini")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Tahminleri görselleştir
visualize_results(model, X_test, y_test)

3. Eğitim Analizi: Loss ve Accuracy Grafikleri
Eğitim sırasında modelin "ezberleyip ezberlemediğini" (overfitting) anlamak için bu grafikleri çizdirmeliyiz.

In [ ]:
def plot_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Loss Grafiği
    ax1.plot(history.history['loss'], label='Train Loss')
    ax1.plot(history.history['val_loss'], label='Val Loss')
    ax1.set_title('Model Kaybı (Loss)')
    ax1.set_xlabel('Epoch')
    ax1.legend()

    # Accuracy Grafiği
    ax2.plot(history.history['accuracy'], label='Train Acc')
    ax2.plot(history.history['val_accuracy'], label='Val Acc')
    ax2.set_title('Model Doğruluğu (Accuracy)')
    ax2.set_xlabel('Epoch')
    ax2.legend()

    plt.show()

plot_history(history)